# TP 5 : Tableaux et Complexité

## Petites questions

1. Écrire une fonction `somme` pour calculer la somme des éléments d'un tableau d'entiers.  
2. Écrire une fonction `maximum` pour calculer le maximum d'un tableau d'entiers. On pourra utiliser la fonction `max` renvoyant le maximum de 2 nombres.  
3. Écrire une fonction `list_of_array` transformant un tableau en liste.  
4. Tester si un tableau est trié par ordre croissant.

In [1]:
let somme t = 
    let s = ref 0 in
    for i=0 to (Array.length t)-1 do
    s := !s + t.(i) done ;
    !s
in 
somme [|1;3;5|]

- : int = 9


In [2]:
let maximum t = 
    let m = ref min_int in
    for i=0 to (Array.length t)-1 do
    m := max !m  t.(i) done ;
    !m
;; 
maximum [||],
maximum [|1; -2; 6; -3; 2; 4; -11; 2; 8;-10; 7|]

val maximum : int array -> int = <fun>


- : int * int = (-4611686018427387904, 8)


In [3]:
let list_of_array t = 
    let l = ref [] in
    for i = (Array.length t)-1 downto 0 do
        l := t.(i)::!l 
    done ;
    !l
in
list_of_array [|1;3;5|]

- : int list = [1; 3; 5]


In [21]:
let croissante t = 
    let croiss = ref true in
    for i=0 to Array.length t - 2 do
        if t.(i) > t.(i+1) then
        croiss := false
    done ;
    !croiss
in
croissante [|1;3;5;3|],
croissante [|1;3;3;5|]

- : bool * bool = (false, true)


## Maximum local dans un tableau

Un maximum local dans un tableau `t` est un indice `i` tel que `t.(i) >= t.(i+1)` et `t.(i) >= t.(i-1)`. Pour les extrémités, qu'une seule de ces conditions doit être vérifiée (si `t.(i-1)` ou `t.(i+1)` n'existe pas).  
1. Montrer qu'il existe forcément un minimum local dans `t`.  
2. Écrire une fonction `max_local1` trouvant un maximum local dans un tableau en regardant chaque élément un par un (recherche séquentielle). Quelle est sa complexité ?  
3. Ecrire une fonction `max_local2` faisant la même chose mais avec une méthode par dichotomie (en divisant par 2 la taille du problème à chaque itération), pour avoir une complexité logarithmique.  
*Aide* : soit `t.(m)` le milieu du tableau. Que peut-on dire si `t.(m) < t.(m+1)` ? Si `t.(m) < t.(m-1)` ?

A partir du moment où le tableau n'est pas vide, il a un maximum global, qui est donc également local

In [5]:
(*2*)
let max_local1 t =
    let n = Array.length t in
    let m = ref  (-1) in
    if n > 1 then
        if t.(0) >= t.(1) then m := 0
        else if t.(n-2) <= t.(n-1) then m := n-1
        else for i=1 to n-2 do
            if t.(i-1) <= t.(i) && t.(i) >= t.(i+1) then
            m := i
        done
    else if n = 1 then m := 1 ;
    !m
in
max_local1 [|1;3;5;3|],
max_local1 [|5;3;5;6|],
max_local1 [|1;3;5;6|]

- : int * int * int = (2, 0, 3)


In [6]:
(*3*)
let max_local2 t =
    let n = Array.length t in
    if n <= 1 then 
        if n = 0 then min_int else 0
    else if t.(0) >= t.(1) then 0
    else if t.(n-1) >= t.(n-2) then n-1
    else
        let rec dicho i j =
            let m = (i+j)/2 in
            if t.(m) <= t.(m-1) then dicho i (m-1)
            else if t.(m) <= t.(m-1) then dicho (m+1) j
            else m
        in
        dicho 0 n
in
max_local2 [|1;3;5;3|],
max_local2 [|5;3;5;6|],
max_local2 [|1;3;5;6|]

- : int * int * int = (2, 0, 3)


## Tri par comptage

Écrire une fonction `tri_comptage : ’a array -> unit` qui trie un tableau `t` de taille $n$ dont les entrées sont des entiers entre 0
et $M$ (où $M$ est le maximum de `t`), en complexité O($n + M$).  
Pour cela :  
- Créer un autre tableau `compte` de taille $M + 1$ (avec `Array.make`), initialement rempli de 0
- Remplir `compte` pour que `compte.(i)` contienne le nombre d'apparitions de `i` dans `t`
- Recopier les éléments de `compte` dans `t` dans l'ordre croissant pour obtenir un tableau trié

In [7]:
let tri_comptage t = 
    let m = maximum t in
    let compte = Array.make (m+1) 0 in
    for i=0 to Array.length t -1 do
        compte.(t.(i)) <- compte.(t.(i))+1
    done ;
    let a = ref 0 in
    for i=0 to m do
        for j=0 to compte.(i)-1 do
            t.(!a) <- i ;
            incr a
        done ;
    done ;
in 
let tableau = [|1;3;5;6;2;6;0;3|] in
tri_comptage tableau ; tableau

- : int array = [|0; 1; 2; 3; 3; 5; 6; 6|]


## Tranche maximum

Ecrire une fonction `tranche_max : int array -> int` qui renvoie en O($n$) la somme maximum d'éléments consécutifs d'un tableau de taille $n$. Par exemple, `tranche_max [|1; -2; 6; -3; 2; 4; -8; 7|]` doit renvoyer $9$, correspondant aux éléments 6; -3; 2; 4.  

*Indice* : parcourir le tableau `t` avec une boucle for. Stocker 2 variables `m` et `m_cur` telles que, au $i$ème passage dans la boucle for :  
- `m` contient la somme maximum d'éléments consécutifs parmi les $i$ premiers éléments du tableau
- `m_cur` contient la somme maximum d'éléments consécutifs finissant en `t.(i)`.  
Par exemple, lorsque `tranche_max [|1; -2; 6; -3; 2; 4; -8; 7|]` exécute la $3$ème itération de la boucle `for` (c'est à dire que -3 est considéré), `m` contient 6 (correspondant au seul élément 6) et `m_cur` contient 3 (correspondant à 6; -3).

In [8]:
let tranche_max t = 
    let n = Array.length t in
    if n <> 0 then
    let m = ref t.(0) in
        let m_cur = ref t.(0) in
        for i=1 to n-1 do
            m_cur := max (!m_cur + t.(i)) (t.(i)) ;
            m := max !m_cur !m
        done ;
        !m
    else min_int
in
tranche_max [|1; -2; 6; -3; 2; 4; -11; 2; 8;-10; 7|],
tranche_max [|1; -2; 6; -3; 2; 4; -8; 2; 8;-10; 7|]

- : int * int = (10, 11)


## Inversions dans un tableau

Étant donné un tableau `t`, une inversion de `t` est un couple d'indices $(i, j)$ tels que $i < j$ et `t`.$(i)$ > `t`.$(j)$. Par exemple, `[|4; 1; 5; 2|]` possède 3 inversions: $(0, 1)$, $(0, 3)$ et $(2, 3)$.
1. Écrire une fonction `inv` en O($n^2$) déterminant le nombre d'inversions d'un tableau de taille $n$. Justifier que la complexité est bien O$(n^2$).
2. Écrire une fonction `inv_triee` telle que, si `t1` et `t2` sont deux tableaux triés par ordre croissant, `inv_triee t1 t2` renvoie le nombre de couples $(i, j)$ tels que `t1.(i) > t2.(j)`. La complexité de `inv_triee` doit être linéaire en le nombre total d'éléments de `t1` et `t2`.
3. (Difficile) Écrire une fonction `inv2` telle que, si `t` est un tableau de taille $n$, `inv2 t` renvoie le nombre d'inversions de `t` en complexité O($n\log(n)$).  
On modifiera le tri fusion pour renvoyer le nombre d'inversions en plus de la liste triée.

In [9]:
let inv t =
    let n = Array.length t in
    let inversions = ref 0 in
    for i=0 to n-2 do (*O(n²)*)
        for j = i+1 to n-1 do (*O(n)*)
            if t.(j) > t.(i) then (*O(1)*)
                incr inversions
        done ;
    done ;
    !inversions
in 
inv [|4; 1; 5; 2|]

- : int = 3


In [10]:
let inv_triee t1 t2 = (*O(n1 + n2)*)
    let inversions = ref 0 in 
    let n1, n2 = Array.length t1, Array.length t2 in
    let j = ref 0 in
    for i = 0 to n1 - 1 do (*O(n1)*)
        inversions := !inversions + !j ;
        while !j < n2 && t1.(i) > t2.(!j) do (*O(n2) en plus de la boucle for*)
            incr j ;
            incr inversions 
        done ;
    done ;
    !inversions
in
inv_triee [|4;7;9|] [|1;2;3;4;5;6;7;8;9|]

- : int = 17
